In [31]:
from docopt import docopt
from glob import glob
import os
import subprocess
import pandas as pd
from tqdm import tqdm
import json

In [23]:
cd ~/work/ocaml-workspace/symetric

/home/feser/work/ocaml-workspace/symetric


In [79]:
csg_run = 'runs/llm-csg-numeric'

In [80]:
rows = []
for out in tqdm(glob(csg_run + "/*.out")):
    name_parts = os.path.basename(out).split(".")[0].split("-")
    bench_name = name_parts[1]
    repeat = name_parts[2]
    bench_file = glob(f"bench/cad_ext/**/{bench_name}", recursive=True)[0]
    proc = subprocess.run(["_build/default/bin/equiv_cad.exe", bench_file, out])
    rows.append({"bench": bench_name, "repeat": repeat, "result": proc.returncode})
csg_df = pd.DataFrame(rows)

100%|█████████████████████████████████████████████████████████████████████████████████| 400/400 [00:05<00:00, 76.75it/s]


In [87]:
csg_df_grouped = csg_df.groupby('bench').agg('min')

In [88]:
def print_csg_results(df):
    print('Correct: ', len(df[df['result'] == 0]))
    print('Incorrect: ', len(df[df['result'] == 1]))
    print('Invalid sexp: ', len(df[df['result'] == 2]))
    print('Invalid CSG program: ', len(df[df['result'] == 3]))
    
print_csg_results(csg_df_grouped)

Correct:  0
Incorrect:  40
Invalid sexp:  0
Invalid CSG program:  0


In [85]:
regex_run = 'runs/llm-regex-sketch/'

In [106]:
rows = []
for out in tqdm(glob(regex_run + "/*.out")):
    name_parts = os.path.basename(out).split(".")[0].split("-")
    bench_name = name_parts[1]
    sketch_name = name_parts[2]
    bench_file = f"vendor/regel/exp/so/benchmark/{bench_name}"
    sketch_file = f"vendor/regel/exp/so/sketch/{bench_name}"
    
    with open(sketch_file, 'r') as f:
        sketches = f.readlines()
    sketch = sketches[int(sketch_name)].split(' ', 1)[1]
        
    try:
        with open(out, 'r') as out_f:
            for (repeat, program) in enumerate(json.load(out_f)):
                proc = subprocess.run(["_build/default/bin/check_regex.exe", "-s", sketch, program], stdin=open(bench_file))
                rows.append({"bench": bench_name + '-' + sketch_name, "repeat": repeat, "result": proc.returncode})
    except json.JSONDecodeError:
        print('Warning: Could not read ', out)
regex_df = pd.DataFrame(rows)


 42%|█████████████████████████████████▍                                              | 907/2174 [02:23<02:35,  8.13it/s]

 44%|███████████████████████████████████▎                                            | 961/2174 [02:27<00:23, 52.64it/s]

 60%|███████████████████████████████████████████████▍                               | 1304/2174 [03:21<02:14,  6.46it/s]Error parsing command line:

  unknown flag -?{<num>}concat(?,or(?,<,>),?<num>)

For usage information, run

  check_regex.exe -help

 61%|████████████████████████████████████████████████▏                              | 1327/2174 [03:25<01:50,  7.66it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 2174/2174 [05:35<00:00,  6.48it/s]


In [107]:
regex_grouped_df = regex_df.groupby('bench').agg('min')
regex_grouped_df

,repeat,result
bench,,
1-0,0,2
1-1,0,2
1-10,0,2
1-2,0,2
1-3,0,0
...,...,...
99-5,0,4
99-6,0,4
99-7,0,4


In [108]:
def print_results(df):
    print('Correct: ', len(df[df['result'] == 0]))
    print('Correct examples: ', len(df[df['result'] == 1]))
    print('Correct spec: ', len(df[df['result'] == 2]))
    print('Both incorrect: ', len(df[df['result'] == 3]))
    print('Invalid regex program: ', len(df[df['result'] == 4]))
    
print_results(regex_grouped_df)

Correct:  111
Correct examples:  34
Correct spec:  1135
Both incorrect:  255
Invalid regex program:  612


In [109]:
tower_run = 'runs/llm-tower'

In [116]:
rows = []
for out in tqdm(glob(tower_run + "/*.out")):
    bench_name = os.path.basename(out).split(".")[0].split("-")[1]
    bench_file = f"bench/tower/{bench_name}.sexp"
            
    try:
        with open(out, 'r') as out_f:
            for (repeat, program) in enumerate(json.load(out_f)):
                with open('/tmp/prog', 'w') as f:
                    f.write(program)
                    
                proc = subprocess.run(["_build/default/bin/equiv_tower.exe", bench_file, '/tmp/prog'])
                rows.append({"bench": bench_name, "repeat": repeat, "result": proc.returncode})
    except json.JSONDecodeError:
        print('Warning: Could not read ', out)
tower_df = pd.DataFrame(rows)

100%|███████████████████████████████████████████████████████████████████████████████████| 34/34 [00:05<00:00,  6.23it/s]


In [115]:
tower_grouped_df = tower_df.groupby('bench').agg('min')
tower_grouped_df

,repeat,result
bench,,
test0-24,0,1
test1-24,0,1
test10-24,0,1
test11-24,0,1
test12-24,0,1
test13-24,0,1
test14-24,0,1
test15-24,0,1
test16-24,0,1


In [117]:
def print_results(df):
    print('Correct: ', len(df[df['result'] == 0]))
    print('Incorrect: ', len(df[df['result'] == 1]))
    print('Invalid: ', len(df[df['result'] == 2]))
    
print_results(tower_grouped_df)

Correct:  0
Incorrect:  34
Invalid:  0
